In [3]:
!pip install -q --upgrade keras
!pip install -q --upgrade keras-nlp


In [4]:
import keras
import keras_nlp
import os
os.environ["KERAS_BACKEND"] = 'tensorflow'



In [5]:
keras.mixed_precision.set_global_policy('mixed_float16')

In [6]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
# !# Remove unsupervised examples
!rm -r aclImdb/train/unsup




  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  7209k      0  0:00:11  0:00:11 --:--:-- 8895k


In [7]:
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb/train/unsup': No such file or directory


In [8]:
train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=32)
test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=32)


Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [9]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    'bert_tiny_en_uncased',
    num_classes=2,
    trainable=True)
classifier.fit(train_ds,validation_data=test_ds,epochs=1)

100%|██████████| 507/507 [00:00<00:00, 1.18MB/s]
100%|██████████| 16.8M/16.8M [00:00<00:00, 17.9MB/s]
100%|██████████| 547/547 [00:00<00:00, 384kB/s]
100%|██████████| 226k/226k [00:00<00:00, 940kB/s]


782/782 ━━━━━━━━━━━━━━━━━━━━ 85s 63ms/step - loss: 0.5570 - sparse_categorical_accuracy: 0.6987 - val_loss: 0.3202 - val_sparse_categorical_accuracy: 0.8643


In [10]:
Preprocessor = keras_nlp.models.BertPreprocessor.from_preset('bert_tiny_en_uncased',sequence_length=512)
import tensorflow as tf

In [11]:
train_cached = train_ds.map(Preprocessor,tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
test_cached = test_ds.map(Preprocessor,tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)

In [12]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    'bert_tiny_en_uncased',
    num_classes=2,
    preprocessor = None)
classifier.fit(train_cached,validation_data=test_cached,epochs=3)

Epoch 1/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 65s 52ms/step - loss: 0.5599 - sparse_categorical_accuracy: 0.6916 - val_loss: 0.3148 - val_sparse_categorical_accuracy: 0.8670
Epoch 2/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 39ms/step - loss: 0.3051 - sparse_categorical_accuracy: 0.8728 - val_loss: 0.2929 - val_sparse_categorical_accuracy: 0.8788
Epoch 3/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.2429 - sparse_categorical_accuracy: 0.9036 - val_loss: 0.3077 - val_sparse_categorical_accuracy: 0.8771


In [16]:
tokenizer = keras_nlp.models.BertTokenizer.from_preset('bert_tiny_en_uncased')
tokenizer(["i love this project"])

<tf.RaggedTensor [[1045, 2293, 2023, 2622]]>

In [26]:
packer = keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    sequence_length=64)

def preprocessor(x,y):
  token_ids,segment_ids = packer(tokenizer(x))
  print(token_ids)
  print(segment_ids)
  x= {
       "token_ids":token_ids,
       "segment_ids":segment_ids,
       "padding_mask":token_ids!=0
  }
  return x,y

In [29]:
for text , label in train_ds.take(1):
  x,y=preprocessor(text,label)
  print(y)

tf.Tensor(
[[ 101 1045 2079 ... 2039 1998  102]
 [ 101 1045 2081 ... 2003 2205  102]
 [ 101 2055 1996 ... 1056 2342  102]
 ...
 [ 101 2045 2064 ... 2024 2895  102]
 [ 101 2023 2003 ... 1007 2023  102]
 [ 101 2057 2018 ... 5019 1045  102]], shape=(32, 64), dtype=int32)
tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(32, 64), dtype=int32)
tf.Tensor([0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 1 1 1 0], shape=(32,), dtype=int32)


In [31]:
train_preprocessed = train_ds.map(preprocessor,tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
test_preprocessed = test_ds.map(preprocessor,tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
classifier = keras_nlp.models.BertClassifier.from_preset(
    'bert_tiny_en_uncased',
    num_classes=2,
    preprocessor = None)
classifier.fit(train_preprocessed,validation_data=test_preprocessed,epochs=3)

Tensor("multi_segment_packer_9_1/RaggedToTensor/RaggedTensorToTensor:0", shape=(None, 64), dtype=int32, device=/device:CPU:*)
Tensor("multi_segment_packer_9_1/RaggedToTensor_1/RaggedTensorToTensor:0", shape=(None, 64), dtype=int32, device=/device:CPU:*)


Tensor("multi_segment_packer_9_1/RaggedToTensor/RaggedTensorToTensor:0", shape=(None, 64), dtype=int32, device=/device:CPU:*)
Tensor("multi_segment_packer_9_1/RaggedToTensor_1/RaggedTensorToTensor:0", shape=(None, 64), dtype=int32, device=/device:CPU:*)
Epoch 1/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 71s 49ms/step - loss: 0.6315 - sparse_categorical_accuracy: 0.6294 - val_loss: 0.5082 - val_sparse_categorical_accuracy: 0.7470
Epoch 2/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - loss: 0.4864 - sparse_categorical_accuracy: 0.7709 - val_loss: 0.4895 - val_sparse_categorical_accuracy: 0.7608
Epoch 3/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.4247 - sparse_categorical_accuracy: 0.8068 - val_loss: 0.5156 - val_sparse_categorical_accuracy: 0.7550
